<a href="https://colab.research.google.com/github/zia207/Deep-Neural-Network-Satellite-Image-Classification-in-Google-Colaboratory-iPython-Note-Book-/blob/master/NoteBook/Advance_Regression/02-02-06-poisson-regression-hurdle-r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](http://drive.google.com/uc?export=view&id=1bLQ3nhDbZrCCqy_WCxxckOne2lgVvn3l)

# 6. Hurdle Model

In count data analysis, we often encounter datasets with excessive zeros. These zeros can be problematic for standard Poisson or negative binomial models, as they typically assume that zero counts arise from the same process as positive counts. However, in many cases, zeros and positive counts result from distinct processes. For example, in modeling the number of visits to a gym, some people never visit (true zeros), while others visit at varying frequencies. The hurdle model is designed to handle this by splitting the modeling process into two parts: one to handle zero vs. non-zero outcomes and another to model the positive counts.

This tutorial introduces hurdle models in R, where we’ll start with a model overview, explaining how hurdle models work to separate zero and count components. We’ll then cover how to fit hurdle models from scratch to understand their two-part structure, and we’ll use the {pscl} package for efficient model fitting. Finally, we’ll discuss model comparison techniques, such as the likelihood ratio and Vuong tests, to assess fit quality and evaluate prediction performance on test data to validate the model.

## Overview

A **hurdle model** is a type of statistical model used to analyze count data, particularly when the data has an excess number of zeros (zero-inflated data). It is commonly applied in scenarios where there are two separate processes: one that governs whether an event occurs (i.e., whether the count is zero or non-zero) and another that determines the count value once the event occurs.

***Key Features of the Hurdle Model***

1.  **Two-Part Structure**:
    -   **First part (Hurdle component)**: This models the probability that the count is greater than zero (i.e., whether the "hurdle" is crossed). Typically, this is done using a binary model, like **logistic regression** or **probit regression**.
    -   **Second part (Count component)**: This models the count of events given that the count is greater than zero. Common models for this part include **Poisson** or **negative binomial regression**, depending on the distribution of the non-zero counts.
2.  **Zero-Inflation Handling**:
    -   The hurdle model is specifically designed to address data where zeros are more frequent than would be expected from standard count models, like Poisson or negative binomial models. This makes it useful in contexts like healthcare (e.g., number of hospital visits), economics (e.g., number of purchases), or environmental science (e.g., number of rare species observed).

***Example***

Consider data on the number of times people visit a doctor in a year. Many people may not visit a doctor at all (zeros), while others visit a variable number of times. In this case: - The **first part** of the hurdle model would predict the likelihood that someone visits the doctor at least once. - The **second part** would model the number of visits, conditional on having at least one visit.

***Difference from Zero-Inflated Models***

Although both hurdle models and zero-inflated models address excess zeros, they are conceptually different: - **Hurdle models** assume a zero outcome is a result of a distinct process (e.g., some people never visit the doctor). - **Zero-inflated models** assume there are two types of zeros: "structural zeros" (those who never visit the doctor) and "sampling zeros" (those who could visit but didn't in a given period).

As we discuss before, the hurdle model consists of two components:

1\. **Binary Component**: A **logistic (or binomial) regression** that models whether the count is zero or non-zero (i.e., whether the hurdle is crossed).

This part of the model predicts whether the outcome is zero or positive.

$$ P(Y_i = 0 | X) = \frac{1}{1 + \exp(X\beta)} $$

Where:

$Y_i = 0$ indicates that the count is zero.

\- $X$ represents the predictor variables.

\- $\beta$ represents the coefficients estimated by logistic regression.

This component uses logistic regression to estimate the probability of zeros (whether or not the "hurdle" is crossed).

2\. **Truncated Count Component** (Positive count): Once the hurdle has been crossed (i.e., we have a positive count), the second part of the model predicts the non-zero counts. This part models only the positive counts and ignores the zeros.

For a **truncated Poisson regression**, the probability mass function is:

$$ P(Y_i = k | Y_i > 0, X) = \frac{\lambda^k e^{-\lambda}}{k!(1 - e^{-\lambda})} $$

Where: -

$k > 0$ represents the non-zero counts.

\- $\lambda = \exp(X\gamma)$ is the mean of the Poisson distribution, with $X$ as the predictors and $\gamma$) as the coefficients.

\- $e^{-\lambda}$ is the probability of zero in the untruncated Poisson model.

For a **truncated Negative Binomial regression**, the equation would incorporate the dispersion parameter $\theta$ to handle overdispersion:

$$ P(Y_i = k | Y_i > 0, X) = \frac{\Gamma(k + \theta^{-1})}{k! \Gamma(\theta^{-1})} \left( \frac{\lambda}{\lambda + \theta^{-1}} \right)^k \left( \frac{\theta^{-1}}{\lambda + \theta^{-1}} \right)^{\theta^{-1}} $$

Where: - $\theta$ is the dispersion parameter that accounts for overdispersion in the data. - $\gamma$ is the Gamma function.

The full likelihood for the Hurdle Model combines the two parts:

$$  P(Y_i | X) = \begin{cases}
P(Y_i = 0 | X) & \text{if } Y_i = 0 \\
P(Y_i = k | Y_i > 0, X) & \text{if } Y_i > 0
\end{cases} $$

## Install rpy2

In [ ]:
!pip uninstall rpy2 -y
!pip install rpy2==3.5.1
%load_ext rpy2.ipython

Found existing installation: rpy2 3.4.2
Uninstalling rpy2-3.4.2:
  Successfully uninstalled rpy2-3.4.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rpy2: filename=rpy2-3.5.1-cp310-cp310-linux_x86_64.whl size=314952 sha256=1181730a2c820200419c834db66e26d69047423745bbf18c5e012a7ac5b29bc3
  Stored in directory: /root/.cache/pip/wheels/73/a6/ff/4e75dd1ce1cfa2b9a670cbccf6a1e41c553199e9b25f05d953
Successfully built rpy2


## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Check and install required R packages

In [ ]:
%%R
pkg <- c('tidyverse',
     'plyr',
		 'gt',
		 'DataExplorer',
		 'rstatix',
		 'gtsummary',
		 'report',
		 'performance',
		 'jtools',
		 'margins',
		 'marginaleffects',
		 'ggeffects',
		 'patchwork',
		 'Metrics',
		 'ggpmisc',
		 'caret',
		 'tidymodels',
		 'metrica',
		 'RColorBrewer',
    'MASS',
    'epiDisplay',
		 'pscl',
		 'VGAM',
		 'AER'
		  )
new.packages <- pkg[!(pkg %in% installed.packages(lib='drive/My Drive/R/')[,"Package"])]
if(length(new.packages)) install.packages(new.packages, lib='drive/My Drive/R/')

## Fit a Hurdle model

To fit a hurdle model manually in R using synthetic data without any R packages, you will implement both the binary and count components of the hurdle model from scratch. Below, I provide a complete example, including data generation, model fitting for the zero and count parts, and predictions.

### Data Generation

The counts variable simulates a zero-inflated count variable using Poisson distribution and introduces excess zeros based on a probability.

In [ ]:
%%R
# Step 1: Generate Synthetic Data
set.seed(42)
n <- 200  # Number of observations
x <- rnorm(n)  # Independent variable
lambda <- exp(0.5 * x)  # Rate for Poisson distribution
zero_inflation_prob <- 0.7  # Probability of excess zeros

# Generate counts based on the zero-inflated process
counts <- rpois(n, lambda)  # Poisson distributed counts
counts <- ifelse(runif(n) < zero_inflation_prob, 0, counts)  # Introduce excess zeros

# Combine into a data frame
data <- data.frame(counts = counts, x = x)
head(data)

  counts          x
1      0  1.3709584
2      0 -0.5646982
3      1  0.3631284
4      0  0.6328626
5      0  0.4042683
6      0 -0.1061245


### Fit a Binary Model: Implement logistic regression manually to predict whether the count is zero or non-zero.

In [ ]:
%%R
# Step 2: Fit the Binary Model (Logistic Regression)
# Model to predict whether counts are greater than zero
binary_model_fit <- function(x, y) {
  # Create the logistic function
  logistic <- function(beta0, beta1, x) {
    1 / (1 + exp(-(beta0 + beta1 * x)))
  }

  # Initialize coefficients
  beta <- c(0, 0)  # beta0, beta1
  learning_rate <- 0.01
  n_iterations <- 1000

  # Gradient Descent to optimize beta
  for (i in 1:n_iterations) {
    p_hat <- logistic(beta[1], beta[2], x)

    # Calculate the gradients
    grad_beta0 <- sum((y - p_hat) * p_hat * (1 - p_hat)) / length(y)
    grad_beta1 <- sum((y - p_hat) * p_hat * (1 - p_hat) * x) / length(y)

    # Update the coefficients
    beta[1] <- beta[1] + learning_rate * grad_beta0
    beta[2] <- beta[2] + learning_rate * grad_beta1
  }
  return(beta)
}

# Fit binary model
binary_response <- as.numeric(data$counts > 0)  # 1 if counts > 0, else 0
binary_coefficients <- binary_model_fit(data$x, binary_response)

### Fit a Count Model: Implement Poisson regression manually for positive counts.

In [ ]:
%%R
# Step 3: Fit the Count Model (Poisson Regression)
# Only use positive counts for fitting
poisson_model_fit <- function(x, y) {
  # Create the log link function for Poisson regression
  log_link <- function(beta0, beta1, x) {
    beta0 + beta1 * x
  }

  # Initialize coefficients
  beta <- c(0, 0)  # beta0, beta1
  learning_rate <- 0.01
  n_iterations <- 1000

  # Gradient Descent to optimize beta
  for (i in 1:n_iterations) {
    lambda_hat <- exp(log_link(beta[1], beta[2], x))

    # Calculate the gradients
    grad_beta0 <- sum((y - lambda_hat) / lambda_hat) / length(y)
    grad_beta1 <- sum((y - lambda_hat) / lambda_hat * x) / length(y)

    # Update the coefficients
    beta[1] <- beta[1] + learning_rate * grad_beta0
    beta[2] <- beta[2] + learning_rate * grad_beta1
  }
  return(beta)
}

# Fit Poisson model using positive counts
positive_counts <- data[data$counts > 0, ]
poisson_coefficients <- poisson_model_fit(positive_counts$x, positive_counts$counts)

### Combine Predictions: Use the predictions from both models to get the final predicted counts.

In [ ]:
%%R
# Step 4: Combine Predictions
# Predict probabilities of non-zero counts
logistic_function <- function(beta0, beta1, x) {
  1 / (1 + exp(-(beta0 + beta1 * x)))
}
data$prob_nonzero <- logistic_function(binary_coefficients[1], binary_coefficients[2], data$x)

# Predict expected counts for positive cases
data$expected_counts <- ifelse(data$counts > 0, exp(poisson_coefficients[1] + poisson_coefficients[2] * data$x), 0)

# Combine predictions for final results
data$hurdle_predicted_counts <- ifelse(data$counts > 0, data$expected_counts * data$prob_nonzero, 0)

# View results
head(data)

# Summary statistics
summary(data)

     counts            x             prob_nonzero    expected_counts
 Min.   :0.000   Min.   :-2.99309   Min.   :0.2665   Min.   :0.000  
 1st Qu.:0.000   1st Qu.:-0.61011   1st Qu.:0.3295   1st Qu.:0.000  
 Median :0.000   Median :-0.01643   Median :0.3463   Median :0.000  
 Mean   :0.285   Mean   :-0.02748   Mean   :0.3465   Mean   :0.283  
 3rd Qu.:0.000   3rd Qu.: 0.63363   3rd Qu.:0.3652   3rd Qu.:0.000  
 Max.   :6.000   Max.   : 2.70189   Max.   :0.4278   Max.   :3.228  
 hurdle_predicted_counts
 Min.   :0.0000         
 1st Qu.:0.0000         
 Median :0.0000         
 Mean   :0.1034         
 3rd Qu.:0.0000         
 Max.   :1.2960         


## Fit a Hurdle model in R

To fit a hurdle model using the `hurdle()` function from the {pscl} package in R with the NMES1988 dataset, follow below steps:



### Check and Install Required R Packages

In [ ]:
%%R
pkg <- c('tidyverse',
     'plyr',
		 'gt',
		 'DataExplorer',
		 'rstatix',
		 'gtsummary',
		 'report',
		 'performance',
		 'jtools',
		 'margins',
		 'marginaleffects',
		 'ggeffects',
		 'patchwork',
		 'Metrics',
		 'ggpmisc',
		 'caret',
		 'tidymodels',
		 'metrica',
		 'RColorBrewer',
    'MASS',
    'epiDisplay',
		 'pscl',
		 'VGAM',
		 'AER'
		  )
new.packages <- pkg[!(pkg %in% installed.packages(lib='drive/My Drive/R/')[,"Package"])]
if(length(new.packages)) install.packages(new.packages, lib='drive/My Drive/R/')

### Load R pacakges

In [ ]:
%%R
# set library path
.libPaths('drive/My Drive/R')
library(tidyverse)
library(plyr)
library(gt)
library(rstatix)
library(gtsummary)
library(DataExplorer)
library(report)
library(performance)
library(jtools)
library(margins)
library(marginaleffects)
library(ggeffects)
library(patchwork)
library(Metrics)
library(ggpmisc)
library(RColorBrewer)
library(MASS)
library(epiDisplay)
library(dlookr)
library(pscl)
library(VGAM)
library(AER)

### Data

We will use physician office visits data set (**NMES1988**) from [AER](https://cran.r-project.org/web/packages/AER/index.html) package. . It represents a sample of 4,406 individuals aged 66 and over who were covered by Medicare in 1988. One of the variables in the data is the number of physician office visits. If we want to create a model for the number of visits using some of the other variables in the dataset, we need to start by loading the data. You may also need to install the AER package.

A data frame containing 4,406 observations on 19 variables. We will use following variables:

-   visits- Number of physician office visits.
-   hospital - Number of hospital stays.
-   health - Factor indicating self-perceived health status, levels are "poor", "average" (reference category), "excellent".
-   chronic - Number of chronic conditions.
-   age - Age in years (divided by 10).
-   afam - Factor. Is the individual African-American?
-   gender - Factor indicating gender.
-   married- Factor. is the individual married?
-   school - Number of years of education.
-   income- Family income in USD 10,000.
-   employed - Factor. Is the individual employed?
-   insurance- Factor. Is the individual covered by private insurance?
-   medicaid Factor - Is the individual covered by Medicaid?

In [ ]:
%%R
# load the data
data(NMES1988)
# select variables
df<-NMES1988 |>
  dplyr::select(visits,
                hospital,
                health,
                chronic,
                age,
                afam,
                gender,
                married,
                school,
                income,
                employed,
                insurance,
                medicaid)

# split  data
seeds = 11076
tr_prop = 0.70

# training data (70% data)
train= ddply(df,.(gender, afam),
                 function(., seed) { set.seed(seed); .[sample(1:nrow(.), trunc(nrow(.) * tr_prop)), ] }, seed = 101)
test = ddply(df, .(gender, afam),
            function(., seed) { set.seed(seed); .[-sample(1:nrow(.), trunc(nrow(.) * tr_prop)), ] }, seed = 101)


### Fit a hurdle Model

Now, we'll fit a hurdle model using `hurdle()` function of   {pscl} package. We'll model the binary part (whether or not the person visited the doctor) and the count part (how many visits, given that they visited). We will use  `dist =  "poisson"` for the data with Piosson distribution.

In [ ]:
%%R
# Fit the hurdle model using the pscl package
fit.hurdle.pois <- hurdle(
              visits  ~
                hospital +
                health +
                chronic +
                age +
                afam +
                gender +
                married +
                school +
                income +
                employed +
                insurance +
                medicaid,
                data = train,
                 zero.dist = "binomial",
                 dist =  "poisson")
# Summary of the hurdle model
summary(fit.hurdle.pois)


Call:
hurdle(formula = visits ~ hospital + health + chronic + age + afam + 
    gender + married + school + income + employed + insurance + medicaid, 
    data = train, dist = "poisson", zero.dist = "binomial")

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-4.5649 -1.1383 -0.4630  0.5496 19.3241 

Count model coefficients (truncated poisson with log link):
                 Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.920082   0.106057  18.104  < 2e-16 ***
hospital         0.152069   0.007313  20.794  < 2e-16 ***
healthpoor       0.214664   0.021576   9.949  < 2e-16 ***
healthexcellent -0.361636   0.038574  -9.375  < 2e-16 ***
chronic          0.091637   0.005860  15.637  < 2e-16 ***
age             -0.067319   0.012989  -5.183 2.19e-07 ***
afamyes         -0.013614   0.027013  -0.504  0.61428    
gendermale      -0.057532   0.017816  -3.229  0.00124 ** 
marriedyes      -0.074486   0.018247  -4.082 4.46e-05 ***
school           0.016566   0.002357   7.028 

Alternatively, you could use negative binomial distribution if overdispersion is present. Here  use `dist = "negbin" ` if you expect overdispersion in the count data.

In [ ]:
%%R
# Fit the hurdle model using the pscl package
fit.hurdle.nb <- hurdle(
                visits  ~
                hospital +
                health +
                chronic +
                age +
                afam +
                gender +
                married +
                school +
                income +
                employed +
                insurance +
                medicaid,
                data = train,
                zero.dist = "binomial",
                dist =  "negbin")
# Summary of the hurdle model
summary(fit.hurdle.nb)


Call:
hurdle(formula = visits ~ hospital + health + chronic + age + afam + 
    gender + married + school + income + employed + insurance + medicaid, 
    data = train, dist = "negbin", zero.dist = "binomial")

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-1.1992 -0.7122 -0.2742  0.3281 13.3177 

Count model coefficients (truncated negbin with log link):
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.7525249  0.2597264   6.748 1.50e-11 ***
hospital         0.2097844  0.0254244   8.251  < 2e-16 ***
healthpoor       0.2715317  0.0562376   4.828 1.38e-06 ***
healthexcellent -0.4137847  0.0785668  -5.267 1.39e-07 ***
chronic          0.1127509  0.0148089   7.614 2.66e-14 ***
age             -0.0688877  0.0321429  -2.143 0.032100 *  
afamyes         -0.0264495  0.0652791  -0.405 0.685348    
gendermale      -0.0520124  0.0428769  -1.213 0.225105    
marriedyes      -0.1016623  0.0446715  -2.276 0.022859 *  
school           0.0181822  0.0054619


-   The **binary part** of the hurdle model estimates the factors associated with the likelihood of visiting the doctor at all.

-   The **count part** estimates the factors associated with the number of doctor visits for those who visited at least once.

You can inspect the coefficients and interpret their significance:

-   A positive coefficient in the binary part (logistic regression) means the predictor increases the likelihood of visiting the doctor.

-   A positive coefficient in the count part means the predictor increases the number of visits among those who visit the doctor.

We know that each component of a hurdle model can have different sets of predictors. We can do this in the `hurdle(0`) function by using "\|" in the model formula. For example, let's say we want to fit the zero hurdle component using only the insurance and gender predictors. We can do that as follows:

Model below fist the count data model (visits regressed on all other variables) conditional on the zero hurdle model (visits regressed on gender and insurance).

In [ ]:
%%R
fit.hurdle.nb.01 <- hurdle(
                visits ~ . | gender + insurance,
                data = train,
                zero.dist = "binomial",
                dist =  "negbin")
summary(fit.hurdle.nb.01)


Call:
hurdle(formula = visits ~ . | gender + insurance, data = train, dist = "negbin", 
    zero.dist = "binomial")

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-1.1094 -0.7330 -0.2616  0.3502 13.2554 

Count model coefficients (truncated negbin with log link):
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)      1.7525249  0.2597264   6.748 1.50e-11 ***
hospital         0.2097844  0.0254244   8.251  < 2e-16 ***
healthpoor       0.2715317  0.0562376   4.828 1.38e-06 ***
healthexcellent -0.4137847  0.0785668  -5.267 1.39e-07 ***
chronic          0.1127509  0.0148089   7.614 2.66e-14 ***
age             -0.0688877  0.0321429  -2.143 0.032100 *  
afamyes         -0.0264495  0.0652791  -0.405 0.685348    
gendermale      -0.0520124  0.0428769  -1.213 0.225105    
marriedyes      -0.1016623  0.0446715  -2.276 0.022859 *  
school           0.0181822  0.0054619   3.329 0.000872 ***
income          -0.0006644  0.0071340  -0.093 0.925799    
employedyes  

### Model Comparison

#### Likelihood Ratio Test (LRT)

In [ ]:
%%R
# 1. Extract log-likelihoods
logLik.hurdle.pois <- logLik(fit.hurdle.pois)
logLik.hurdle.nb<- logLik(fit.hurdle.nb)
# 2. Compute the LRT statistic
lrt_stat <- 2 * (logLik.hurdle.nb- logLik.hurdle.pois)
# 3. Degrees of freedom difference (df)
df_diff <- attr(logLik.hurdle.nb, "df") - attr(logLik.hurdle.pois, "df")
# 4. Compute p-value (chi-squared distribution)
p_value <- pchisq(lrt_stat, df = df_diff, lower.tail = FALSE)
# Output LRT statistic and p-value
cat("LRT Statistic:", lrt_stat, "\n")
cat("Degrees of Freedom Difference:", df_diff, "\n")
cat("p-value:", p_value, "\n")

LRT Statistic: 5192.7 
Degrees of Freedom Difference: 1 
p-value: 0 


#### Vuong test

In [ ]:
%%R
vuong(fit.hurdle.nb,fit.hurdle.pois)

Vuong Non-Nested Hypothesis Test-Statistic: 
(test-statistic is asymptotically distributed N(0,1) under the
 null that the models are indistinguishible)
-------------------------------------------------------------
              Vuong z-statistic             H_A    p-value
Raw                    11.86906 model1 > model2 < 2.22e-16
AIC-corrected          11.86906 model1 > model2 < 2.22e-16
BIC-corrected          11.86906 model1 > model2 < 2.22e-16


### Prediction Performance

The `predict()` function will be used to predict the number of diabetes patients the test counties. This will help to validate the accuracy of the these regression models.

In [ ]:
%%R
# Prediction
test$Pred.hurdle.pois<-predict(fit.hurdle.pois, test, type = "response")
test$Pred.hurdle.nb<-predict(fit.hurdle.nb, test, type = "response")

# RMSE
rmse.pois<-Metrics::rmse(test$visits, test$Pred.hurdle.pois)
rmse.nb<-Metrics::rmse(test$visits, test$Pred.hurdle.nb)

cat("Hurdlermse Poisson:" ,rmse.pois, "\n")
cat("Hurdlermse NB:", rmse.nb, "\n")

Hurdlermse Poisson: 6.977813 
Hurdlermse NB: 7.040287 


## Summary and Conclusions

In this tutorial, we explored the hurdle model in R, a powerful tool for analyzing count data with an excess of zeros, commonly found in healthcare, economics, and other fields. We applied the hurdle model using both Poisson and negative binomial distributions to model two key processes: whether an individual visits the doctor at all and, if they do, how often they visit.

By fitting and interpreting the hurdle model in R, we've demonstrated how to address complex zero-inflated count data scenarios, providing a clearer understanding of the relationships between predictors and outcomes. This method can be extended to other real-world problems where excess zeros and over-dispersion are prevalent.

## References

1.  [Hurdle Model](https://m-clark.github.io/models-by-example/hurdle.html)

2.  [Getting Started with Hurdle Models](https://library.virginia.edu/data/articles/getting-started-with-hurdle-models)